In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 글꼴깨짐 방지
import matplotlib
import matplotlib.font_manager as fm

import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

fm.get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

In [2]:
df = pd.read_csv('C:/Users/Playdata/Downloads/235745_parking_data/train.csv')

In [3]:
df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [4]:
def preprocessing(df):
    # 오류 단지코드가 존재하는 행들을  사전에 제거
    df_error =  ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
    #df_error =  ['C2335', 'C1327']
    df = df[~df['단지코드'].isin(df_error)].reset_index(drop=True)
    df.rename(columns = {'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철','도보 10분거리 내 버스정류장 수':'버스'},inplace=True)
    df.drop(columns=['임대보증금','임대료','자격유형','임대건물구분'],axis = 1,inplace=True)
    지역_비율 = (df.groupby(['지역'])['총세대수'].count())/(df.groupby(['지역'])['총세대수'].count().sum())*100
    지역_비율=지역_비율.reset_index(name='지역_비율')
    공급유형_비율 = (df.groupby(['공급유형'])['총세대수'].count())/(df.groupby(['공급유형'])['총세대수'].count().sum())*100
    공급유형_비율=공급유형_비율.reset_index(name='공급유형_비율')
    df = pd.merge(df,지역_비율, on='지역')
    df = pd.merge(df,공급유형_비율, on='공급유형')
    df.drop(columns=['지역','공급유형','단지코드'],axis = 1,inplace=True)
    df=df.dropna(axis=0)
    df = df[['총세대수', '전용면적', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수', '공급유형_비율',
       '지역_비율', '등록차량수']]
    return df

In [5]:
df = preprocessing(df)

In [6]:
!pip install XGBoost

Defaulting to user installation because normal site-packages is not writeable


In [7]:
from sklearn.model_selection import train_test_split
X = df.iloc[:, 1:-1]
y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
X

,전용면적,전용면적별세대수,공가수,지하철,버스,단지내주차면수,공급유형_비율,지역_비율
0,33.48,276,17.0,0.0,3.0,624.0,59.944751,12.741713
1,39.60,60,17.0,0.0,3.0,624.0,59.944751,12.741713
2,39.60,20,17.0,0.0,3.0,624.0,59.944751,12.741713
3,46.90,38,17.0,0.0,3.0,624.0,59.944751,12.741713
4,46.90,19,17.0,0.0,3.0,624.0,59.944751,12.741713
...,...,...,...,...,...,...,...,...
2891,51.99,80,9.0,0.0,3.0,1590.0,0.310773,2.209945
2892,59.93,142,9.0,0.0,3.0,1590.0,0.310773,2.209945
2893,51.32,126,13.0,0.0,3.0,673.0,0.310773,2.209945
2894,59.88,49,13.0,0.0,3.0,673.0,0.310773,2.209945


In [261]:
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

from time import time

from xgboost import XGBRegressor

In [262]:
import xgboost as xgb

In [263]:
model = xgb.XGBRegressor()
start = time()
model.fit(X_train, y_train)
train_time = time() - start
start = time()
y_pred = model.predict(X_test)
predict_time = time()-start    
print(model)
print("\tTraining time: %0.3fs" % train_time)
print("\tPrediction time: %0.3fs" % predict_time)
print("\tExplained variance:", explained_variance_score(y_test, y_pred))
print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tR2 score:", r2_score(y_test, y_pred))
print()

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)
	Training time: 0.162s
	Prediction time: 0.006s
	Explained variance: 0.9821692578560057
	Mean absolute error: 25.76014966982481
	R2 score: 0.9821289337052278



In [264]:
import joblib

joblib.dump(model, 'XGBoostingRegressor.pkl')

['XGBoostingRegressor.pkl']

In [265]:
XGBRegressor = joblib.load('XGBoostingRegressor.pkl')
pred = XGBRegressor.predict(X)
pred

array([189.52061, 213.52315, 227.39891, ..., 642.9397 , 706.8685 ,
       720.988  ], dtype=float32)

In [268]:
from sklearn.model_selection import GridSearchCV

grid = xgb.XGBRegressor(n_estimators=100,
                        learning_rate= 0.15,
                        max_depth =3,
                        gamma= 0, 
                        colsample_bytree = 0.9, 
                        eval_metric= 'mae',
                        scale_pos_weight=0.5, 
                        subsample=0.5,
                        min_child_weight =20,
                        seed = 1337, 
                        reg_alpha= 0.01, 
                        reg_lambda=0.1)
                         
grid.fit(X_train, y_train)
pred_re = grid.predict(X_test)

In [269]:
from sklearn.metrics import accuracy_score, mean_absolute_error
mean_absolute_error(y_test, pred_re)

83.50457291274542